# AE on proteins in SA representation

In [1]:
import glob
import os
from collections import Counter
import string
from keras import Input
from keras.layers import Dense, Lambda, Conv1D
import keras.backend as K
from keras.models import Model
from keras.objectives import binary_crossentropy, mse
import os
import random
import numpy as np
import NotebookLoader
from keras.optimizers import RMSprop, Adam
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import load_model
from tempfile import TemporaryFile
import csv

Using TensorFlow backend.


In [2]:
import Preprocessing as pre

importing Jupyter notebook from Preprocessing.ipynb


### Configs

In [3]:
lengths = {}
for f in pre.families:
    proteins = glob.glob(os.path.join(pre.family_paths[f], "*.out"))
    print("Proteins for family %s" %f)
    for p in proteins:
        print(p)
    lengths[f] = len(p)
total = sum([lengths[f] for f in pre.families])

Proteins for family fam_1
../dataset/families/fam_1/2EBN.lf_str.out
../dataset/families/fam_1/1JFX.lf_str.out
../dataset/families/fam_1/1ITX.lf_str.out
../dataset/families/fam_1/1NAR.lf_str.out
../dataset/families/fam_1/1KFW.lf_str.out
../dataset/families/fam_1/1B1Y.lf_str.out
../dataset/families/fam_1/1VFF.lf_str.out
../dataset/families/fam_1/1EDG.lf_str.out
../dataset/families/fam_1/1CNV.lf_str.out
Proteins for family fam_2
../dataset/families/fam_2/1HLB.lf_str.out
../dataset/families/fam_2/2LHB.lf_str.out
../dataset/families/fam_2/1DLW.lf_str.out
../dataset/families/fam_2/1ITH.lf_str.out
../dataset/families/fam_2/1ECA.lf_str.out
../dataset/families/fam_2/1ASH.lf_str.out
../dataset/families/fam_2/1MBA.lf_str.out
../dataset/families/fam_2/2HBG.lf_str.out
Proteins for family fam_3
../dataset/families/fam_3/2SAS.lf_str.out
../dataset/families/fam_3/1OMR.lf_str.out
../dataset/families/fam_3/1SRA.lf_str.out
../dataset/families/fam_3/1CB1.lf_str.out
../dataset/families/fam_3/1IQ3.lf_str.ou

In [4]:
padding = True
num_classes = 25
categorical = False
use_angles = True
max_length = 668
flatten = True

In [5]:
batch_size = 64
intermediate_dim = 25
epochs = 20
learning_rate = 0.001

### Build the autoencoder

In [6]:
def create_checkpoints(f):
    checkpoints_path = os.path.join("models", f)
    tensorboard_path = os.path.join("logs", f)
    cp_cb = ModelCheckpoint(filepath=os.path.join(checkpoints_path, "model" + f + ".hdf5"), monitor='val_loss',
                            save_best_only=True)
    tb_cb = TensorBoard(log_dir=tensorboard_path)
    return [cp_cb, tb_cb]

In [7]:
#autoencoder
def get_ae():
    if categorical:
        if not flatten:
            model_input = Input(shape=(None,num_classes))
        else:
            model_input = Input(shape=(max_length*num_classes,))
    elif use_angles:
        model_input = Input(shape=(max_length*3,))
    else:
        model_input = Input(shape=(max_length,))
    #x=Conv1D(intermediate_dim, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1)(model_input)
    #encoded=Conv1D(intermediate_dim, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1, name="encoded")(x)
    #x=Conv1D(num_classes, activation='sigmoid', kernel_size=3, padding='same', dilation_rate=1)(encoded)
    encoded= Dense(intermediate_dim, activation='sigmoid')(model_input)
    if categorical:
        if not flatten:
            x = Dense(num_classes, activation='sigmoid')(encoded)
        else:
            x = Dense(max_length*num_classes, activation='sigmoid')(encoded)
    elif use_angles:
        x = Dense(max_length*3, activation='sigmoid')(encoded)
    else:
        x = Dense(max_length, activation='sigmoid')(encoded)
    ae=Model(inputs=model_input, outputs=[x])
    opt=RMSprop(lr=learning_rate)
    ae.compile(optimizer=opt, loss='binary_crossentropy', metrics=['mean_absolute_error'])
    ae.summary()
    return ae

### Train and evaluate the autoencoder for specific classes

In [8]:
models = {"fam_1": "models/fam_1/modelfam_1.hdf5",
          "fam_2": "models/fam_2/modelfam_2.hdf5",
          "fam_3": "models/fam_3/modelfam_3.hdf5",
          "fam_4": "models/fam_4/modelfam_4.hdf5",
          "fam_5": "models/fam_5/modelfam_5.hdf5",
          "fam_6": "models/fam_6/modelfam_6.hdf5",
          "fam_7": "models/fam_7/modelfam_7.hdf5",
          "fam_8": "models/fam_8/modelfam_8.hdf5",
          "fam_9": "models/fam_9/modelfam_9.hdf5"}

In [9]:
path = '/tmp/data_serialized_proteins'
#for r in range(1, 10):
#    os.makedirs(os.path.join(path, 'fam_%d' % r))

In [10]:
def evaluate_for_fam(f):
    print("Test for autoencoder on fam %s" %f)
    train_filename = os.path.join(path, f, "train.npy")
    train = np.load(train_filename)
    ae = load_model(models[f])
    ae.summary()
    losses_train = []
    for t in train:
        losses_train.append(ae.evaluate(np.array([t]),np.array([t]), verbose=0)[1])
    max_l = max(losses_train)
    losses_test = []
    del train
    tp, tn, fp, fn = 0, 0, 0, 0
    tp_p, tn_p, fp_p, fn_p = 0, 0, 0, 0
    for ft in pre.families:
        print("Test for fam %s" %ft)
        test_filename = os.path.join(path, ft, "test.npy")
        test = np.load(test_filename)
        for t in test:
            loss=ae.evaluate(np.array([t]),np.array([t]), verbose=0)[1]
            if loss > max_l:
                # predict other family
                if ft == f:
                    fn+=1
                else:
                    tn+=1
            else:
                # predict current family
                if ft == f:
                    tp+=1 
                else:
                    fp+=1
            # compute the probability
            if loss > max_l:
                pr = 1 - (max_l / (2 * loss))
            else:
                pr = loss / (2 * max_l)
            if pr >= 0.5:
                # predict other family
                if ft == f:
                    fn_p+=1
                else:
                    tn_p+=1
            else:
                # predict current family
                if ft == f:
                    tp_p+=1 
                else:
                    fp_p+=1  
        del test
    return [tp, tn, fp, fn], [tp_p, tn_p, fp_p, fn_p]

In [11]:
# %%capture output

res_fam = open('res_fam_conf.csv', mode='w')
res_avg = open('res_avg_conf.csv', mode='w')
writer_fam = csv.writer(res_fam, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
writer_fam.writerow(['Iteration', 'Superfamily', 'TP', 'TN', 'FP', 'FN', 'Prec', 'Recall', 'Spec', 'AUC'])
writer_avg = csv.writer(res_avg, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
writer_avg.writerow(['Iteration', 'Prec', 'Recall', 'Spec', 'AUC'])
for i in range(0,20):
    for f in pre.families:
        families_conf = pre.load_family(f)
        test_size = int(0.25 * len(families_conf))
        val_size =  int(0.15 * len(families_conf))
        train_all, test = train_test_split(families_conf, test_size=test_size, random_state=i)
        del families_conf
        train, val = train_test_split(train_all, test_size = val_size, random_state=i)
        del train_all
        train_filename = os.path.join(path, f, "train.npy")
      
        train = pre.process_conf(train, categorical=categorical, use_angles=use_angles, padding=padding, max_length=max_length, flatten=flatten)  
        print("train: " + repr(train.shape))
        np.save(train_filename, train)
        del train
        
        val = pre.process_conf(val, categorical=categorical, use_angles=use_angles, padding=padding, max_length=max_length, flatten=flatten)  
        print("val: " + repr(val.shape))
        val_filename = os.path.join(path, f, "val.npy")
        np.save(val_filename, val)
        del val
        
        test = pre.process_conf(test, categorical=categorical, use_angles=use_angles, padding=padding, max_length=max_length, flatten=flatten)  

        print("test: " + repr(test.shape))
        test_filename = os.path.join(path, f, "test.npy")
        np.save(test_filename, test)
        del test
        
    for f in pre.families:
        print("Training for family %s" %f)
        train_filename = os.path.join(path, f, "train.npy")
        test_filename = os.path.join(path, f, "val.npy")
        train = np.load(train_filename)
        test = np.load(test_filename)
        print("train: " + repr(train.shape))
        print("test" + repr(test.shape))
        ae = get_ae()
        ae.fit(train, train,
               shuffle=True,
               epochs=epochs,
               batch_size=batch_size,
               validation_data=(test, test),
               callbacks=create_checkpoints(f),
               verbose=1)    
        del train
        del test
    # evaluate
    tp, tn, fp, fn = {}, {}, {}, {}
    tp_p, tn_p, fp_p, fn_p = {}, {}, {}, {}
    prec, recall, spec, auc = {}, {}, {}, {}
    prec_p, recall_p, spec_p, auc_p = {}, {}, {}, {}
    for f in pre.families:
        print("Evaluating family %s" %f)
        [tp[f], tn[f], fp[f], fn[f]], [tp_p[f], tn_p[f], fp_p[f], fn_p[f]] = evaluate_for_fam(f)
        prec_p[f] = (1.0* tp_p[f] / (tp_p[f] + fp_p[f]))
        recall_p[f] = (1.0* tp_p[f] / (tp_p[f] + fn_p[f]))
        spec_p[f] = (1.0* tn_p[f] / (tn_p[f] + fp_p[f]))
        auc_p[f] = (recall_p[f] + spec_p[f]) / 2
        # write to csv 
        print([i, f, tp_p[f], tn_p[f], fp_p[f], tn_p[f], prec_p[f], recall_p[f], spec_p[f], auc_p[f]])
        writer_fam.writerow([i, f, tp_p[f], tn_p[f], fp_p[f], tn_p[f], prec_p[f], recall_p[f], spec_p[f], auc_p[f]])
    prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p = 0, 0, 0, 0
    for f in pre.families:
        prec_wavg_p += lengths[f] * prec_p[f] / total
        recall_wavg_p += lengths[f] * recall_p[f] / total
        spec_wavg_p += lengths[f] * spec_p[f] / total
        auc_wavg_p += lengths[f] * auc_p[f] / total
    # write to csv
    print([i, prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p])
    writer_avg.writerow([i, prec_wavg_p, recall_wavg_p, spec_wavg_p, auc_wavg_p])
res_fam.close()
res_avg.close()

Proteins for family fam_1
../dataset/families/fam_1/2EBN.lf_str.out
../dataset/families/fam_1/1JFX.lf_str.out
../dataset/families/fam_1/1ITX.lf_str.out
../dataset/families/fam_1/1NAR.lf_str.out
../dataset/families/fam_1/1KFW.lf_str.out
../dataset/families/fam_1/1B1Y.lf_str.out
../dataset/families/fam_1/1VFF.lf_str.out
../dataset/families/fam_1/1EDG.lf_str.out
../dataset/families/fam_1/1CNV.lf_str.out
Counter({282: 10000, 214: 10000, 416: 10000, 286: 10000, 432: 10000, 497: 10000, 420: 10000, 377: 10000, 280: 10000})
(54000, 668, 3)
(54000, 2004)
train: (54000, 2004)
(13500, 668, 3)
(13500, 2004)
val: (13500, 2004)
(22500, 668, 3)
(22500, 2004)
test: (22500, 2004)
Proteins for family fam_2
../dataset/families/fam_2/1HLB.lf_str.out
../dataset/families/fam_2/2LHB.lf_str.out
../dataset/families/fam_2/1DLW.lf_str.out
../dataset/families/fam_2/1ITH.lf_str.out
../dataset/families/fam_2/1ECA.lf_str.out
../dataset/families/fam_2/1ASH.lf_str.out
../dataset/families/fam_2/1MBA.lf_str.out
../datas

54000/54000 [==============================] - 6s 102us/step - loss: 0.3402 - mean_absolute_error: 0.0895 - val_loss: 0.3349 - val_mean_absolute_error: 0.0826
Epoch 14/20
54000/54000 [==============================] - 6s 102us/step - loss: 0.3304 - mean_absolute_error: 0.0794 - val_loss: 0.3263 - val_mean_absolute_error: 0.0762
Epoch 15/20
54000/54000 [==============================] - 6s 104us/step - loss: 0.3242 - mean_absolute_error: 0.0742 - val_loss: 0.3214 - val_mean_absolute_error: 0.0713
Epoch 16/20
54000/54000 [==============================] - 6s 103us/step - loss: 0.3206 - mean_absolute_error: 0.0707 - val_loss: 0.3185 - val_mean_absolute_error: 0.0699
Epoch 17/20
54000/54000 [==============================] - 5s 102us/step - loss: 0.3185 - mean_absolute_error: 0.0687 - val_loss: 0.3161 - val_mean_absolute_error: 0.0671
Epoch 18/20
54000/54000 [==============================] - 6s 103us/step - loss: 0.3171 - mean_absolute_error: 0.0675 - val_loss: 0.3183 - val_mean_absolute_

36000/36000 [==============================] - 4s 98us/step - loss: 0.1473 - mean_absolute_error: 0.0345 - val_loss: 0.1465 - val_mean_absolute_error: 0.0338
Epoch 12/20
36000/36000 [==============================] - 4s 97us/step - loss: 0.1463 - mean_absolute_error: 0.0336 - val_loss: 0.1455 - val_mean_absolute_error: 0.0333
Epoch 13/20
36000/36000 [==============================] - 4s 97us/step - loss: 0.1455 - mean_absolute_error: 0.0330 - val_loss: 0.1455 - val_mean_absolute_error: 0.0334
Epoch 14/20
36000/36000 [==============================] - 4s 97us/step - loss: 0.1450 - mean_absolute_error: 0.0324 - val_loss: 0.1445 - val_mean_absolute_error: 0.0320
Epoch 15/20
36000/36000 [==============================] - 4s 99us/step - loss: 0.1446 - mean_absolute_error: 0.0320 - val_loss: 0.1441 - val_mean_absolute_error: 0.0317
Epoch 16/20
36000/36000 [==============================] - 4s 100us/step - loss: 0.1443 - mean_absolute_error: 0.0316 - val_loss: 0.1438 - val_mean_absolute_error

30000/30000 [==============================] - 3s 102us/step - loss: 0.2256 - mean_absolute_error: 0.0137 - val_loss: 0.2272 - val_mean_absolute_error: 0.0134
Epoch 10/20
30000/30000 [==============================] - 3s 103us/step - loss: 0.2253 - mean_absolute_error: 0.0135 - val_loss: 0.2271 - val_mean_absolute_error: 0.0135
Epoch 11/20
30000/30000 [==============================] - 3s 102us/step - loss: 0.2252 - mean_absolute_error: 0.0133 - val_loss: 0.2270 - val_mean_absolute_error: 0.0139
Epoch 12/20
30000/30000 [==============================] - 3s 102us/step - loss: 0.2250 - mean_absolute_error: 0.0132 - val_loss: 0.2270 - val_mean_absolute_error: 0.0138
Epoch 13/20
30000/30000 [==============================] - 3s 102us/step - loss: 0.2250 - mean_absolute_error: 0.0132 - val_loss: 0.2267 - val_mean_absolute_error: 0.0130
Epoch 14/20
30000/30000 [==============================] - 3s 103us/step - loss: 0.2249 - mean_absolute_error: 0.0132 - val_loss: 0.2268 - val_mean_absolute_

36000/36000 [==============================] - 4s 101us/step - loss: 0.1878 - mean_absolute_error: 0.0173 - val_loss: 0.1865 - val_mean_absolute_error: 0.0167
Epoch 8/20
36000/36000 [==============================] - 4s 101us/step - loss: 0.1870 - mean_absolute_error: 0.0161 - val_loss: 0.1859 - val_mean_absolute_error: 0.0157
Epoch 9/20
36000/36000 [==============================] - 4s 101us/step - loss: 0.1864 - mean_absolute_error: 0.0151 - val_loss: 0.1854 - val_mean_absolute_error: 0.0148
Epoch 10/20
36000/36000 [==============================] - 4s 100us/step - loss: 0.1860 - mean_absolute_error: 0.0144 - val_loss: 0.1850 - val_mean_absolute_error: 0.0140
Epoch 11/20
36000/36000 [==============================] - 4s 99us/step - loss: 0.1856 - mean_absolute_error: 0.0138 - val_loss: 0.1846 - val_mean_absolute_error: 0.0135
Epoch 12/20
36000/36000 [==============================] - 4s 101us/step - loss: 0.1854 - mean_absolute_error: 0.0132 - val_loss: 0.1844 - val_mean_absolute_err

...

Train on 36000 samples, validate on 9000 samples
Epoch 1/20
36000/36000 [==============================] - 10s 283us/step - loss: 0.2182 - mean_absolute_error: 0.0951 - val_loss: 0.1807 - val_mean_absolute_error: 0.0618
Epoch 2/20
36000/36000 [==============================] - 6s 162us/step - loss: 0.1805 - mean_absolute_error: 0.0621 - val_loss: 0.1807 - val_mean_absolute_error: 0.0620
Epoch 3/20
36000/36000 [==============================] - 6s 161us/step - loss: 0.1804 - mean_absolute_error: 0.0621 - val_loss: 0.1794 - val_mean_absolute_error: 0.0612
Epoch 4/20
36000/36000 [==============================] - 6s 159us/step - loss: 0.1741 - mean_absolute_error: 0.0576 - val_loss: 0.1694 - val_mean_absolute_error: 0.0540
Epoch 5/20
36000/36000 [==============================] - 6s 163us/step - loss: 0.1654 - mean_absolute_error: 0.0511 - val_loss: 0.1619 - val_mean_absolute_error: 0.0485
Epoch 6/20
36000/36000 [==============================] - 6s 163us/step - loss: 0.1590 - mean_absolu

Train on 30000 samples, validate on 7500 samples
Epoch 1/20
30000/30000 [==============================] - 9s 296us/step - loss: 0.3237 - mean_absolute_error: 0.1221 - val_loss: 0.2470 - val_mean_absolute_error: 0.0430
Epoch 2/20
30000/30000 [==============================] - 5s 163us/step - loss: 0.2392 - mean_absolute_error: 0.0303 - val_loss: 0.2346 - val_mean_absolute_error: 0.0245
Epoch 3/20
30000/30000 [==============================] - 5s 163us/step - loss: 0.2338 - mean_absolute_error: 0.0225 - val_loss: 0.2315 - val_mean_absolute_error: 0.0208
Epoch 4/20
30000/30000 [==============================] - 5s 165us/step - loss: 0.2312 - mean_absolute_error: 0.0195 - val_loss: 0.2295 - val_mean_absolute_error: 0.0184
Epoch 5/20
30000/30000 [==============================] - 5s 163us/step - loss: 0.2295 - mean_absolute_error: 0.0176 - val_loss: 0.2281 - val_mean_absolute_error: 0.0170
Epoch 6/20
30000/30000 [==============================] - 5s 164us/step - loss: 0.2283 - mean_absolut

Train on 36000 samples, validate on 9000 samples
Epoch 1/20
36000/36000 [==============================] - 10s 274us/step - loss: 0.2889 - mean_absolute_error: 0.1232 - val_loss: 0.2247 - val_mean_absolute_error: 0.0643
Epoch 2/20
36000/36000 [==============================] - 6s 163us/step - loss: 0.2109 - mean_absolute_error: 0.0456 - val_loss: 0.2003 - val_mean_absolute_error: 0.0318
Epoch 3/20
36000/36000 [==============================] - 6s 162us/step - loss: 0.1962 - mean_absolute_error: 0.0267 - val_loss: 0.1926 - val_mean_absolute_error: 0.0227
Epoch 4/20
36000/36000 [==============================] - 6s 160us/step - loss: 0.1906 - mean_absolute_error: 0.0200 - val_loss: 0.1886 - val_mean_absolute_error: 0.0174
Epoch 5/20
36000/36000 [==============================] - 6s 159us/step - loss: 0.1876 - mean_absolute_error: 0.0159 - val_loss: 0.1865 - val_mean_absolute_error: 0.0144
Epoch 6/20
36000/36000 [==============================] - 6s 162us/step - loss: 0.1860 - mean_absolu

Train on 30000 samples, validate on 7500 samples
Epoch 1/20
30000/30000 [==============================] - 9s 296us/step - loss: 0.3729 - mean_absolute_error: 0.1425 - val_loss: 0.3254 - val_mean_absolute_error: 0.1012
Epoch 2/20
30000/30000 [==============================] - 5s 163us/step - loss: 0.3120 - mean_absolute_error: 0.0893 - val_loss: 0.3018 - val_mean_absolute_error: 0.0800
Epoch 3/20
30000/30000 [==============================] - 5s 162us/step - loss: 0.2965 - mean_absolute_error: 0.0753 - val_loss: 0.2904 - val_mean_absolute_error: 0.0702
Epoch 4/20
30000/30000 [==============================] - 5s 164us/step - loss: 0.2873 - mean_absolute_error: 0.0665 - val_loss: 0.2830 - val_mean_absolute_error: 0.0625
Epoch 5/20
30000/30000 [==============================] - 5s 163us/step - loss: 0.2812 - mean_absolute_error: 0.0601 - val_loss: 0.2783 - val_mean_absolute_error: 0.0578
Epoch 6/20
30000/30000 [==============================] - 5s 164us/step - loss: 0.2774 - mean_absolut

Test for fam fam_1
Test for fam fam_2
Test for fam fam_3
Test for fam fam_4
Test for fam fam_5
Test for fam fam_6
Test for fam fam_7
Test for fam fam_8
Test for fam fam_9
[19, 'fam_5', 12500, 125000, 0, 125000, 1.0, 1.0, 1.0, 1.0]
Evaluating family fam_6
Test for autoencoder on fam fam_6
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_177 (InputLayer)       (None, 2004)              0         
_________________________________________________________________
dense_353 (Dense)            (None, 25)                50125     
_________________________________________________________________
dense_354 (Dense)            (None, 2004)              52104     
Total params: 102,229
Trainable params: 102,229
Non-trainable params: 0
_________________________________________________________________
Test for fam fam_1
Test for fam fam_2
Test for fam fam_3
Test for fam fam_4
Test for fam fam_5
Test for fam fam